In [1]:
import pandas as pd
from pandasql import sqldf
import pyparsing as pyp
from os import path
import string

INTERSECTION_IN = '../data/raw/Jefferson_County_KY_Street_Intersections.csv'
assert path.exists(INTERSECTION_IN)

XS = pd.read_csv(INTERSECTION_IN)

STOPLIGHTS_IN = '../data/raw/Jefferson_County_KY_Signalized_Intersections.csv'
assert path.exists(STOPLIGHTS_IN)

STOPS = pd.read_csv(STOPLIGHTS_IN)

In [2]:
XS.columns, STOPS.columns

(Index(['X', 'Y', 'OBJECTID', 'SIFCODE1', 'SIFCODE2', 'INTID', 'SCCAD_ID',
        'FST_INTPRE', 'FST_INTNAME', 'FST_INTSUF', 'SEC_INTPRE', 'SEC_INTNAME',
        'SEC_INTSUF', 'X_COORD', 'Y_COORD', 'FST_SIFID', 'SEC_SIFID',
        'GLOBALID'],
       dtype='object'),
 Index(['X', 'Y', 'OBJECTID', 'UNITID', 'SIGID', 'MAINSTREET', 'CROSSSTREET',
        'OWNER2', 'ROUTE', 'MILEPOINT', 'DESCRIPTION', 'TIMES', 'OWNER', 'TYPE',
        'INTID', 'ATMSID'],
       dtype='object'))

In [3]:
XS['INTID'] = XS.INTID.str.strip()
STOPS['INTID'] = STOPS.INTID.str.strip()

In [4]:
# Easy to match
easy_match = XS[XS.INTID.isin(STOPS.INTID)]



In [16]:
easy = sqldf("""SELECT XS.INTID, XS.FST_INTNAME, XS.SEC_INTNAME, STOPS.MAINSTREET, STOPS.CROSSSTREET, STOPS.DESCRIPTION
      FROM XS JOIN STOPS
      ON XS.INTID == STOPS.INTID""")
easy.head()

,INTID,FST_INTNAME,SEC_INTNAME,MAINSTREET,CROSSSTREET,DESCRIPTION
0,16340775940,JUNEAU,SHELBYVILLE,JUNEAU,SHELBYVILLE,JUNEAU @ SHELBYVILLE
1,16790939409,DATA VAULT,ENGLISH STATION,DATA VAULT,ENGLISH STATION,DATA VAULT @ ENGLISH STATION
2,18059409996,SHELBYVILLE,I 265 RAMP,SHELBYVILLE,I 265 RAMP,SHELBYVILLE @ I 265 RAMP
3,28012247132,COLLINS,WESTPORT,COLLINS,WESTPORT,COLLINS @ WESTPORT
4,28110531224,CHAMBERLAIN,COLLINS,CHAMBERLAIN,COLLINS,CHAMBERLAIN @ COLLINS


In [17]:
# Not so easy to match
intid_2 = sqldf("SELECT * FROM STOPS WHERE STOPS.INTID = 2")
intid_2


hard = sqldf("""SELECT XS.INTID, XS.FST_INTNAME, XS.SEC_INTNAME, intid_2.MAINSTREET, intid_2.CROSSSTREET, intid_2.DESCRIPTION
      FROM XS JOIN intid_2
      ON (XS.FST_INTNAME == intid_2.MAINSTREET)
      AND XS.SEC_INTNAME == intid_2.CROSSSTREET""")
hard.head()

,INTID,FST_INTNAME,SEC_INTNAME,MAINSTREET,CROSSSTREET,DESCRIPTION
0,779103840827,BARDSTOWN,BUECHEL,BARDSTOWN,BUECHEL,BARDSTOWN @ BUECHEL
1,824403847291,BARDSTOWN,WATTERSON,BARDSTOWN,WATTERSON,BARDSTOWN @ WATTERSON
2,824403847355,BARDSTOWN,WATTERSON,BARDSTOWN,WATTERSON,BARDSTOWN @ WATTERSON
3,825621905945,FERN VALLEY,SHEPHERDSVILLE,FERN VALLEY,SHEPHERDSVILLE,FERN VALLEY @ SHEPHERDSVILLE
4,840103840751,BARDSTOWN,BUECHEL,BARDSTOWN,BUECHEL,BARDSTOWN @ BUECHEL


In [32]:
def parse_description(description:str):
    main, cross = description.split("@")
    return main.strip(), cross.strip()

def verifier(fst_intname, sec_intname, main_street, cross_street, description):
    desc_main, desc_cross = parse_description(description)
    return all((
        fst_intname == main_street,
        sec_intname == cross_street,
        fst_intname == desc_main,
        sec_intname == desc_cross))

def verify_row(row):
    test = True
    try:
        desc_main, desc_cross = parse_description(row.DESCRIPTION)
    except AttributeError:
        pass
    except ValueError:
        pass
    else:
        test &= row.FST_INTNAME == desc_main
        test &= row.SEC_INTNAME == desc_cross
        
    test &= row.FST_INTNAME == row.MAINSTREET
    test &= row.SEC_INTNAME == row.CROSSSTREET
    
    return test

parse_description("POPLAR LEVEL @ JEFFERSON")
verifier("BARDSTOWN", "BUECHEL", "BARDSTOWN", "BUECHEL", "BARDSTOWN @ BUECHEL")
verify_row(easy.loc[0])

True

In [45]:
easy[easy.apply(verify_row, axis=1) == True] # Most of these work
not_so_easy = easy[easy.apply(verify_row, axis=1) == False]
not_so_easy

,INTID,FST_INTNAME,SEC_INTNAME,MAINSTREET,CROSSSTREET,DESCRIPTION
5,2834756E294,OLD HENRY,TERRA CROSSING,OLD HENRY RD,TERRA CROSSING BLVD,OLD HENRY @ TERRA CROSSING BLVD
13,76547569996,OLD HENRY,I 265 RAMP,OLD HENRY RD,I 265 SB RAMP,OLD HENRY RD @ I 265 SB RAMP
46,181871327354,WESTPORT,WESTPORT,WESTPORT RD,WESTPORT CIR,WESTPORT RD @ WESTPORT CIR
108,433648057132,ORMSBY,WESTPORT,WESTPORT RD,ORMSBY LN,WESTPORT @ ORMSBY
120,460407638870,BUNSEN,BUNSEN,BUNSEN PKY,PLANTSIDE DR,BUNSEN PKY @ PLANTSIDE DR
126,477491259183,HURSTBOURNE,HURSTBOURNE,HURSTBOURNE PKY,HURSTBOURNE TRCE,HURSTBOURNE PKY @ HURSTBOURNE TRCE
155,532306561717,BRECKENRIDGE,DUTCHMANS,BRECKENRIDGE LN,DUTCHMANS LN,BRECKENRIDGE LN @ DUTCHMANS LN
161,545817179463,DUTCHMANS,DUTCHMANS,DUTCHMANS PKY,DUTCHMANS LN,DUTCHMANS PKY @ DUTCHMANS LN
164,560830429126,HURSTBOURNE,HURSTBOURNE,S HURSTBOURNE PKY,HURSTBOURNE,S HURSTBOURNE PKY @ HURSTBOURNE
194,698671329996,WESTPORT,I 265 RAMP,WESTPORT RD,I 265 RAMP,WESTPORT RD @ I 265 RAMP


In [77]:

prefixes = {x for x in XS.FST_INTPRE.str.strip().unique() if x}
prefixes.union([x for x in XS.SEC_INTPRE.str.strip().unique() if x])

suffixes = {s for s in XS.FST_INTSUF.str.strip().unique() if s}
suffixes.union({s for s in XS.SEC_INTSUF.str.strip().unique() if s})

def strip_roadname(roadname:str) -> str:
    for prefix in prefixes:
        new = roadname.lstrip(prefix)
    for suffix in suffixes:
        new = new.strip(suffix)
    return new.strip()

strip_roadname("OLD HENRY RD")


'HENRY'

In [40]:
hard[hard.apply(verify_row, axis=1) == False] # This is empty, which is what I want. 

,INTID,FST_INTNAME,SEC_INTNAME,MAINSTREET,CROSSSTREET,DESCRIPTION


In [99]:
easy_match = sqldf("""SELECT XS.INTID as XS_INTID, XS.FST_INTNAME, XS.SEC_INTNAME,
            STOPS.INTID as STOPS_INTID, STOPS.MAINSTREET, STOPS.CROSSSTREET
      FROM XS JOIN STOPS
      ON XS.INTID = STOPS.INTID""")
start = 100
easy_match[easy_match.FST_INTNAME.isin(easy_match.MAINSTREET)].loc[start:start+10]

,XS_INTID,FST_INTNAME,SEC_INTNAME,STOPS_INTID,MAINSTREET,CROSSSTREET
100,388108272930,BUECHEL,HIKES,388108272930,BUECHEL,HIKES
101,391606566005,BRECKENRIDGE,SIX MILE,391606566005,BRECKENRIDGE,SIX MILE
102,400067127413,U S HIGHWAY 42,WOODSTONE,400067127413,U S HIGHWAY 42,WOODSTONE
103,418544927132,MURPHY,WESTPORT,418544927132,MURPHY,WESTPORT
104,418910537132,CHAMBERLAIN,WESTPORT,418910537132,CHAMBERLAIN,WESTPORT
105,487976248353,BLANKENBAKER,COMMONWEALTH,487976248353,BLANKENBAKER,COMMONWEALTH
106,488719907293,EVERGREEN,WOODLAND,488719907293,EVERGREEN,WOODLAND
107,428707248262,BROWNSBORO,BARBOUR,428707248262,BROWNSBORO,BARBOUR
108,433648057132,ORMSBY,WESTPORT,433648057132,WESTPORT RD,ORMSBY LN
109,435827747132,LYNDON,WESTPORT,435827747132,LYNDON,WESTPORT


In [136]:
names = pd.DataFrame(columns='STOPS XS'.split())
names['STOPS'] = pd.concat((STOPS.MAINSTREET, STOPS.CROSSSTREET)).dropna().unique()

xs_names = pd.concat((XS.FST_INTNAME, XS.SEC_INTNAME)).dropna().unique()
xs_names

for i, data in names.iterrows():
    name = data.STOPS
    match = list()
    for xsname in xs_names:
        if xsname in name:
            match.append(xsname)
    names.at[i, 'XS'] = match

names


,STOPS,XS
0,OLD HENRY,"[OLD HENRY, HENRY, H]"
1,SHELBYVILLE,"[SHELBYVILLE, SHELBY, H]"
2,BROWNSBORO,"[BROWNSBORO, BROWN, BROWNS]"
3,HERR,"[HERR, H]"
4,U S HIGHWAY 42,"[U S HIGHWAY 42, HIGH, H, HIGHWAY 42]"
...,...,...
619,THIXTON LN,"[THIXTON, H]"
620,I 265 SB RAMP,[M]
621,OLD HEADY RD,"[HEADY, OLD HEADY, H]"
622,ORMSBY LN,"[ORMSBY, M]"
